In [ ]:
import cv2
import numpy as np
import os


In [ ]:
# V2
def are_centers_close(center1, center2, threshold=5):
    return np.linalg.norm(np.array(center1) - np.array(center2)) < threshold

def remove_smaller_circles(circles, min_radius=20):
    if circles is None:
        return []

    circles = np.uint16(np.around(circles))
    unique_circles = []

    for current_circle in circles[0, :]:
        add_circle = True
        for unique_circle in unique_circles:
            if are_centers_close(current_circle[:2], unique_circle[:2]):
                if current_circle[2] > unique_circle[2]:
                    # Replace the smaller circle with the larger one
                    unique_circle[:] = current_circle
                add_circle = False
                break
        if add_circle and current_circle[2] >= min_radius:
            unique_circles.append(current_circle)

    return unique_circles

## 1 Biển

In [ ]:
import cv2
import numpy as np
import os

def reviewSign(image_original, filename):
    # Chuyển đổi ảnh từ BGR sang RGB
    img_rgb = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB)
    x, y, _ = img_rgb.shape
    # Chuyển đổi ảnh từ RGB sang HSV
    img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)

    # Định nghĩa phạm vi màu đỏ trong không gian màu HSV
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 100, 100])
    upper_red2 = np.array([180, 255, 255])

    # Tạo mặt nạ cho các vùng màu đỏ
    mask1 = cv2.inRange(img_hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(img_hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)

    # Lọc ra các vùng màu đỏ từ ảnh gốc
    red_regions = cv2.bitwise_and(img_rgb, img_rgb, mask=red_mask)

    # Chuyển đổi ảnh RGB thành BGR để phù hợp với định dạng của OpenCV
    red_regions_bgr = cv2.cvtColor(red_regions, cv2.COLOR_RGB2BGR)

    # Chuyển đổi ảnh thành thang độ xám để tìm đường viền
    gray = cv2.cvtColor(red_regions_bgr, cv2.COLOR_BGR2GRAY)

    # Giảm nhiễu để tránh phát hiện nhầm các hình tròn
    gray = cv2.medianBlur(gray, 5)

    # Tìm kiếm các hình tròn trong ảnh sử dụng HoughCircles
    if (x,y) == (1706, 2560): #m5 
        rows = gray.shape[0]  # minDist
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=rows/8,
                                   param1=50, param2=40, minRadius=100, maxRadius=200)
    elif (x,y) == (526, 800): #m6
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT,
                            1.2, 60, param1=100,
                            param2= 0.85, minRadius=10)    
    
    elif (x,y) == (903, 645): # m8
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT,
                                   2, 30, param1=200,
                                   param2=0.85, minRadius=30)
    elif (x,y) == (1333, 2000): # m10
        rows = gray.shape[0]  # minDist
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp=1.5, minDist=rows/8,
                                   param1=50, param2=60, minRadius=100, maxRadius=330)
    else:
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT,
                                   2, 30, param1=200,
                                   param2=0.85, minRadius=10)
        
    filtered_circles = remove_smaller_circles(circles)
    
    # Vẽ các vòng tròn đã lọc
    if filtered_circles is not None:
        for i in filtered_circles:
            center = (i[0], i[1])
            # Tâm của vòng tròn
            cv2.circle(img_rgb, center, 1, (0, 100, 100), 3)
            # Đường viền của vòng tròn
            radius = i[2]
            cv2.circle(img_rgb, center, radius, (0, 255, 0), 3)

    # plt - RGB , cv2 - BGR
    return cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)


def process_images(input_folder, output_folder, check):
    # Ensure output folder exists
    ver2_folder = os.path.join(output_folder, check)
    if not os.path.exists(ver2_folder):
        os.makedirs(ver2_folder)

    # List all images in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Add other image formats if needed
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            if image is not None:
                processed_image = reviewSign(image, filename)
                output_path = os.path.join(ver2_folder, filename)
                cv2.imwrite(output_path, processed_image)
                print(f"Processed and saved {filename} to {ver2_folder}")

# Define paths
input_folder = r"D:\TDTU_Work\DIP\Final\data"
output_folder = r"D:\TDTU_Work\DIP\Final\testImages"
check = "ver2"
# Process images
process_images(input_folder, output_folder, check)


In [ ]:
import cv2
import numpy as np
import os

def reviewSign2(image_original, filename):
    # Chuyển đổi ảnh từ BGR sang RGB
    img_rgb = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB)
    x, y, _ = img_rgb.shape
    # Chuyển đổi ảnh từ RGB sang HSV
    img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)

    # Định nghĩa phạm vi màu đỏ trong không gian màu HSV
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 100, 100])
    upper_red2 = np.array([180, 255, 255])

    # Tạo mặt nạ cho các vùng màu đỏ
    mask1 = cv2.inRange(img_hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(img_hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)

    # Lọc ra các vùng màu đỏ từ ảnh gốc
    red_regions = cv2.bitwise_and(img_rgb, img_rgb, mask=red_mask)

    # Chuyển đổi ảnh RGB thành BGR để phù hợp với định dạng của OpenCV
    red_regions_bgr = cv2.cvtColor(red_regions, cv2.COLOR_RGB2BGR)

    # Chuyển đổi ảnh thành thang độ xám để tìm đường viền
    gray = cv2.cvtColor(red_regions_bgr, cv2.COLOR_BGR2GRAY)

    # Giảm nhiễu để tránh phát hiện nhầm các hình tròn
    # Tìm kiếm các hình tròn trong ảnh sử dụng HoughCircles
    if (x,y) == (188, 268): #m5 
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT,
                            1.2, 10, param1=100,
                            param2= 0.2, minRadius=20)
    elif (x,y) == (177, 285): #m6
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT,
                            1.5, 10, param1=150,
                            param2= 0.2, minRadius=10)  
    
    elif (x,y) == (532, 765): # m8
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT,
                            1.5, 10, param1=150,
                            param2= 0.7, minRadius=50)
    elif (x,y) == (193, 261): # m10
        gray = cv2.medianBlur(gray, 3)

        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT,
                            1.5, 10, param1=150,
                            param2= 0.5, minRadius=22)
        
    elif (x,y) == (398, 600): # m10
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT,
                            1.5, 10, param1=150,
                            param2= 0.4, minRadius=22)
    
    filtered_circles = remove_smaller_circles(circles)
    
    # Vẽ các vòng tròn đã lọc
    if filtered_circles is not None:
        for i in filtered_circles:
            center = (i[0], i[1])
            # Tâm của vòng tròn
            cv2.circle(img_rgb, center, 1, (0, 100, 100), 2)
            # Đường viền của vòng tròn
            radius = i[2]
            cv2.circle(img_rgb, center, radius, (0, 255, 0), 2)
    # plt - RGB , cv2 - BGR
    return cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)


def process_images2(input_folder, output_folder, check):
    # Ensure output folder exists
    ver2_folder = os.path.join(output_folder, check)
    if not os.path.exists(ver2_folder):
        os.makedirs(ver2_folder)

    # List all images in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Add other image formats if needed
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            if image is not None:
                processed_image = reviewSign2(image, filename)
                output_path = os.path.join(ver2_folder, filename)
                cv2.imwrite(output_path, processed_image)
                print(f"Processed and saved {filename} to {ver2_folder}")

# Define paths
input_folder = r"D:\TDTU_Work\DIP\Final\data\bien2"
output_folder = r"D:\TDTU_Work\DIP\Final\testImages"
check = "ver4"
# Process images
process_images2(input_folder, output_folder, check)


## Combine all

In [1]:
import cv2
import numpy as np
import os

In [9]:
# V2
def are_centers_close(center1, center2, threshold=5):
    return np.linalg.norm(np.array(center1) - np.array(center2)) < threshold

def remove_smaller_circles(circles, min_radius=20):
    if circles is None:
        return []

    circles = np.uint16(np.around(circles))
    unique_circles = []

    for current_circle in circles[0, :]:
        add_circle = True
        for unique_circle in unique_circles:
            if are_centers_close(current_circle[:2], unique_circle[:2]):
                if current_circle[2] > unique_circle[2]:
                    # Replace the smaller circle with the larger one
                    unique_circle[:] = current_circle
                add_circle = False
                break
        if add_circle and current_circle[2] >= min_radius:
            unique_circles.append(current_circle)

    return unique_circles

def extract_sign(image, center, radius):
    # Tạo mặt nạ hình tròn
    mask = np.zeros(image.shape[:2], dtype="uint8")
    cv2.circle(mask, center, radius, 255, -1)
    
    # Áp dụng mặt nạ hình tròn lên ảnh gốc
    masked_image = cv2.bitwise_and(image, image, mask=mask)
    
    # Cắt phần ảnh chứa khuôn mặt
    x, y = center
    cropped_sign = masked_image[y-radius:y+radius, x-radius:x+radius]
    
    return cropped_sign

In [17]:
def reviewSign(image_original, filename):
    # Chuyển đổi ảnh từ BGR sang RGB
    img_rgb = cv2.cvtColor(image_original, cv2.COLOR_BGR2RGB)
    x, y, _ = img_rgb.shape
    # Chuyển đổi ảnh từ RGB sang HSV
    img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)

    # Định nghĩa phạm vi màu đỏ trong không gian màu HSV
    lower_red1 = np.array([0, 100, 100])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 100, 100])
    upper_red2 = np.array([180, 255, 255])

    # Tạo mặt nạ cho các vùng màu đỏ
    mask1 = cv2.inRange(img_hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(img_hsv, lower_red2, upper_red2)
    red_mask = cv2.bitwise_or(mask1, mask2)

    # Lọc ra các vùng màu đỏ từ ảnh gốc
    red_regions = cv2.bitwise_and(img_rgb, img_rgb, mask=red_mask)

    # Chuyển đổi ảnh RGB thành BGR để phù hợp với định dạng của OpenCV
    red_regions_bgr = cv2.cvtColor(red_regions, cv2.COLOR_RGB2BGR)

    # Chuyển đổi ảnh thành thang độ xám để tìm đường viền
    gray = cv2.cvtColor(red_regions_bgr, cv2.COLOR_BGR2GRAY)

    # Giảm nhiễu để tránh phát hiện nhầm các hình tròn
    grayBlur = cv2.medianBlur(gray, 5)

    # Tìm kiếm các hình tròn trong ảnh sử dụng HoughCircles
    if (x,y) == (1706, 2560): #m5 
        rows = gray.shape[0]  # minDist
        circles = cv2.HoughCircles(grayBlur, cv2.HOUGH_GRADIENT, dp=1.2, minDist=rows/8,
                                   param1=50, param2=40, minRadius=100, maxRadius=200)
    elif (x,y) == (526, 800): #m6
        circles = cv2.HoughCircles(grayBlur, cv2.HOUGH_GRADIENT_ALT,
                            1.2, 60, param1=100,
                            param2= 0.85, minRadius=10)    
    
    elif (x,y) == (903, 645): # m8
        circles = cv2.HoughCircles(grayBlur, cv2.HOUGH_GRADIENT_ALT,
                                   2, 30, param1=200,
                                   param2=0.85, minRadius=30)
    elif (x,y) == (1333, 2000): # m10
        rows = grayBlur.shape[0]  # minDist
        circles = cv2.HoughCircles(grayBlur, cv2.HOUGH_GRADIENT, dp=1.5, minDist=rows/8,
                                   param1=50, param2=60, minRadius=100, maxRadius=330)
    #### Biển 2
    elif (x,y) == (188, 268):  
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT,
                            1.2, 10, param1=100,
                            param2= 0.2, minRadius=20)
    elif (x,y) == (177, 285): 
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT,
                            1.5, 10, param1=150,
                            param2= 0.2, minRadius=10)  
    elif (x,y) == (532, 765): 
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT,
                            1.5, 10, param1=150,
                            param2= 0.7, minRadius=50)
    elif (x,y) == (193, 261): 
        gray = cv2.medianBlur(gray, 3)

        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT,
                            1.5, 10, param1=150,
                            param2= 0.5, minRadius=22)
    elif (x,y) == (398, 600): 
        circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT_ALT,
                            1.5, 10, param1=150,
                            param2= 0.4, minRadius=22)
    else:
        circles = cv2.HoughCircles(grayBlur, cv2.HOUGH_GRADIENT_ALT,
                                   2, 30, param1=200,
                                   param2=0.85, minRadius=10)
    
    filtered_circles = remove_smaller_circles(circles)
    print(filtered_circles, x, y)
    # Vẽ các vòng tròn đã lọc
    if filtered_circles is not None:
        for j,i in enumerate(filtered_circles):
            center = (i[0], i[1])
            # Tâm của vòng tròn
            # cv2.circle(img_rgb, center, 1, (0, 100, 100), 3)
            # Đường viền của vòng tròn
            radius = i[2]
            cv2.circle(img_rgb, center, radius, (0, 255, 0), 3)
            
            sign = extract_sign(img_rgb, center, radius)
            base_name, extension = os.path.splitext(filename)
            
            newSign = base_name + "_" + str(j+1) + extension
            
            pathSave = r"D:\TDTU_Work\DIP\Final\data\Sign"
            output_path = os.path.join(pathSave, newSign)
            
            cv2.imwrite(output_path,  cv2.cvtColor(sign, cv2.COLOR_RGB2BGR) )
            
    # plt - RGB , cv2 - BGR
    return cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)


def process_images(input_folder, output_folder, check):
    # Ensure output folder exists
    ver2_folder = os.path.join(output_folder, check)
    if not os.path.exists(ver2_folder):
        os.makedirs(ver2_folder)
    
    # List all images in the input folder
    for filename in os.listdir(input_folder):
        print(filename)
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Add other image formats if needed
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            if image is not None:
                processed_image = reviewSign(image, filename)
                output_path = os.path.join(ver2_folder, filename)
                cv2.imwrite(output_path, processed_image)
                print(f"Processed and saved {filename} to {ver2_folder}")

# Define paths
input_folder = r"D:\TDTU_Work\DIP\Final\data\NewData"
output_folder = r"D:\TDTU_Work\DIP\Final\testImages"
check = "ver5"
# Process images
process_images(input_folder, output_folder, check)


m1.jpg
[array([900, 216, 109], dtype=uint16)] 809 1080
Processed and saved m1.jpg to D:\TDTU_Work\DIP\Final\testImages\ver5
m10.jpg
[array([985, 689, 321], dtype=uint16)] 1333 2000
Processed and saved m10.jpg to D:\TDTU_Work\DIP\Final\testImages\ver5
m11.jpg
[array([176,  55,  24], dtype=uint16), array([230,  54,  24], dtype=uint16)] 188 268
Processed and saved m11.jpg to D:\TDTU_Work\DIP\Final\testImages\ver5
m12.jpg
[array([242,  27,  24], dtype=uint16), array([243,  78,  24], dtype=uint16)] 177 285
Processed and saved m12.jpg to D:\TDTU_Work\DIP\Final\testImages\ver5
m13.jpg
[array([387, 106,  66], dtype=uint16), array([519, 104,  68], dtype=uint16)] 532 765
Processed and saved m13.jpg to D:\TDTU_Work\DIP\Final\testImages\ver5
m14.jpg
[array([219,  54,  28], dtype=uint16), array([132,  33,  27], dtype=uint16)] 193 261
Processed and saved m14.jpg to D:\TDTU_Work\DIP\Final\testImages\ver5
m15.jpg
[array([492,  87,  86], dtype=uint16), array([393, 165,  23], dtype=uint16)] 398 600
Proc